only one window?



In [ ]:
var importer = require('../Core');

var closeAllTabs = (keep) => {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
}

var closeAllWindows = (keep) => {
    return client.windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
}

var onlyOneWindow = () => {
    return client
        .windowHandles()
        .then(r => client.window(r.value[0]))
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
    // TODO: close all tabs
}
module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');

const rows = 6;
const columns = 6;
var screen;

var openUrl = (url) => {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=' + width + ',height=' + height);
        }, url, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
    /*
    // TODO: use this openSPA()
    var id = Math.random().toString(36).substring(7);
    return client
        .execute((url, id, width, height) => {
            var a = document.createElement('a');
            a.setAttribute('onclick', 'window.open("' + url + '", "", "width=' + width + ',height=' + height + '"); return false;');
            a.setAttribute('id', id);
            a.href = url;
            a.style.position = 'absolute';
            a.style.zIndex = 4294967295;
            a.style.top = 0;
            a.style.left = 0;
            a.style.bottom = 0;
            a.style.right = 0;
            document.body.appendChild(a);
            setTimeout(() => a.remove(), 200);
        }, url, id, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .click('a[id="' + id + '"]')
        */
}

var createNewWindows = (urls, offset = 0) => {
    var hwnd, handles;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => handles = r.value)
        .then(() => {
            // -1 first window is already used by sliced url
            const count = urls.length - (handles.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    return openUrl(urls[i])
                             .window(hwnd)
                             .then(h => resolve(h))
                             .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .windowHandles()
        .then(r => handles = r.value)
        // position all windows after opening
        .then(() => {
            const promises = [];
            for(var i = 0; i < handles.length; i++) {
                promises.push((i => resolve => {
                    const newPosition = {
                        x: Math.floor((i + offset) % columns)
                            * Math.floor(screen.width / columns),
                        y: Math.floor((i + offset) / columns % rows)
                            * Math.floor(screen.height / rows)
                    };
                    console.log(newPosition);
                    return client
                        .window(handles[i])
                        .windowHandlePosition(newPosition)
                        .then(() => resolve())
                        .catch(e => {console.log(e); resolve()})
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var tileWindows = (urls, offset = 0) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandleMaximize(hwnd)
        .windowHandleSize()
        .then(r => screen = r.value)
        .windowHandlePosition({x: 0, y: 0})
        .then(() => client.windowHandleSize({
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1), offset))
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [9]:
var importer = require('../Core');

var query = 'list of search engines';

$$.async();
importer.import('child process')
    //.then(execCmd => Promise.all([
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"'),
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"')
    //]))
    importer.import('selenium cell')
        .then(runSeleniumCell => runSeleniumCell('tile chrome windows'))
        .then(tileWindows => tileWindows([
            'https://www.google.com/search?q=' + query,
            'https://www.bing.com/search?q=' + query,
            'https://search.yahoo.com/search?p=' + query,
            'https://www.ask.com/web?q=' + query,
            'https://search.aol.com/aol/search?q=' + query,
            'https://www.baidu.com/s?wd=' + query,
            'https://www.wolframalpha.com/input/?i=' + query,
            'https://duckduckgo.com/?q=' + query,
            'https://archive.org/search.php?query=' + query,
        ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
[ '250a905ca6dfd9603be23fcbfbdb1b3c' ]
{ x: 0, y: 0 }
{ x: 1360, y: 0 }
{ x: 2720, y: 0 }
{ x: 4080, y: 0 }
{ x: 5440, y: 0 }
{ x: 6800, y: 0 }
{ x: 0, y: 1016 }
{ x: 1360, y: 1016 }
{ x: 2720, y: 1016 }


{ sessionId: '250a905ca6dfd9603be23fcbfbdb1b3c',
  status: 0,
  value: 
   [ 'CDwindow-4edc9f17-b945-4d2f-9d9c-8e981a906a42',
     'CDwindow-5424fc4a-c25d-41c0-bb6c-abcd93ee59ef',
     'CDwindow-3300d5e8-1c44-4d87-9079-cdfbcecc1143',
     'CDwindow-6ad40f6e-f848-40e2-a8d4-7c92be1d970a',
     'CDwindow-8127d8c6-66e1-4813-9cdd-6142582ca5f8',
     'CDwindow-d657dc47-0af5-45c6-a73a-897b4aa19dbb',
     'CDwindow-bc9d06c9-78d2-4e85-9cd9-25776e35b3fb',
     'CDwindow-57aebde3-5167-45c8-b1ed-38301e0e2ca5',
     'CDwindow-e4714c51-0239-4fe5-90ca-816916d07199' ] }